# Tarea 03: Examen ETL: SPARK 12+1/02

* Curso: ETL
* Nombre: Mayra Goicochea Neyra
* Fecha: 17/12/2019

Se podrá utilizar toda la información que se encuentra en el campus. 

Se va a trabajar sobre varios ficheros de datos:

Usuarios: id_usuario::sexo::edad::id_profesion::codigo_postal

Peliculas: id_pelicula::titulo (año)::tipo1|tipo2|tipo3

Ratings: id_pelicula::id_usuario::puntuacion::fecha_timestamp

A cada una de las preguntas hay que responder explicando brevemente que se pretende hacer antes de lanzar el código.

Al documento lo llamareís con vuestro nombre y apellido. Debeís enviarlo a mi correo de CUNEF antes del final del examen.

El lenguaje para trabajar con Spark podrá ser python o R indistintamente.

In [1]:
# Nota: Carga de las puntuaciones
# Función para parsear la fecha

from datetime import datetime
dateparse = lambda x: datetime.fromtimestamp(float(x))

## Primera tarea: Inicializar spark context y cargar los datos desde los ficheros.

In [2]:
import re

In [3]:
from pyspark import SparkContext
sc = SparkContext()

In [4]:
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark.sql.types import *
import datetime as dt

In [5]:
from pyspark.sql import SQLContext
sqlContext = SQLContext(sc)
from pyspark.sql import Row

In [6]:
def parseMovie(line):
    return line.replace(')(1',') (1').replace(' (1','::1').replace(' (2','::2').replace(')','').split("::")

In [7]:
def parseRating(line):
    return line.split("::")

In [8]:
movies = sc.textFile('movies.dat.txt')

### Carga Dataset Movies.dat
Se crea el contexto SQL a traves del dataframe con los campos Movie_id, Movie_title, Year, Category.

In [10]:
movies_df = movies.map(lambda x: Row(movie_id = int(parseMovie(x)[0]),
                                     movie_title = str(parseMovie(x)[1]),
                                     year = int(parseMovie(x)[2]),
                                     category = str(parseMovie(x)[3]))).toDF()
movies_df.show(20)

+--------------------+--------+--------------------+----+
|            category|movie_id|         movie_title|year|
+--------------------+--------+--------------------+----+
|Animation|Childre...|       1|           Toy Story|1995|
|Adventure|Childre...|       2|             Jumanji|1995|
|      Comedy|Romance|       3|    Grumpier Old Men|1995|
|        Comedy|Drama|       4|   Waiting to Exhale|1995|
|              Comedy|       5|Father of the Bri...|1995|
|Action|Crime|Thri...|       6|                Heat|1995|
|      Comedy|Romance|       7|             Sabrina|1995|
|Adventure|Children's|       8|        Tom and Huck|1995|
|              Action|       9|        Sudden Death|1995|
|Action|Adventure|...|      10|           GoldenEye|1995|
|Comedy|Drama|Romance|      11|American Presiden...|1995|
|       Comedy|Horror|      12|Dracula: Dead and...|1995|
|Animation|Children's|      13|               Balto|1995|
|               Drama|      14|               Nixon|1995|
|Action|Advent

In [12]:
movies_df.registerTempTable("Movies")
movies_df.printSchema()

root
 |-- category: string (nullable = true)
 |-- movie_id: long (nullable = true)
 |-- movie_title: string (nullable = true)
 |-- year: long (nullable = true)



In [13]:
movies_df.select('year').describe().show()

+-------+------------------+
|summary|              year|
+-------+------------------+
|  count|              3883|
|   mean|1986.0669585372134|
| stddev|16.895690157525554|
|    min|              1919|
|    max|              2000|
+-------+------------------+



### Carga Dataset Rating.dat
Se crea el contexto SQL a traves del dataframe con los campos User_id, Movie_id, Rating, Timestamp.

In [14]:
ratings = sc.textFile('ratings.dat.txt')

In [15]:
ratings.map(lambda x: parseRating(x)).take(5)

[['1', '1193', '5', '978300760'],
 ['1', '661', '3', '978302109'],
 ['1', '914', '3', '978301968'],
 ['1', '3408', '4', '978300275'],
 ['1', '2355', '5', '978824291']]

In [16]:
ratings_df = ratings.map(lambda x: Row(user_id = int(parseRating(x)[0]),
                                     movie_id = int(parseRating(x)[1]),
                                     rating = int(parseRating(x)[2]),
                                     timestamp = dateparse(parseRating(x)[3]))).toDF()
ratings_df.show(20)

+--------+------+--------------------+-------+
|movie_id|rating|           timestamp|user_id|
+--------+------+--------------------+-------+
|    1193|     5|2000-12-31 22:12:...|      1|
|     661|     3|2000-12-31 22:35:...|      1|
|     914|     3|2000-12-31 22:32:...|      1|
|    3408|     4|2000-12-31 22:04:...|      1|
|    2355|     5|2001-01-06 23:38:...|      1|
|    1197|     3|2000-12-31 22:37:...|      1|
|    1287|     5|2000-12-31 22:33:...|      1|
|    2804|     5|2000-12-31 22:11:...|      1|
|     594|     4|2000-12-31 22:37:...|      1|
|     919|     4|2000-12-31 22:22:...|      1|
|     595|     5|2001-01-06 23:37:...|      1|
|     938|     4|2000-12-31 22:29:...|      1|
|    2398|     4|2000-12-31 22:38:...|      1|
|    2918|     4|2000-12-31 22:35:...|      1|
|    1035|     5|2000-12-31 22:29:...|      1|
|    2791|     4|2000-12-31 22:36:...|      1|
|    2687|     3|2001-01-06 23:37:...|      1|
|    2018|     4|2000-12-31 22:29:...|      1|
|    3105|   

In [17]:
ratings_df.registerTempTable("Rating")
ratings_df.printSchema()

root
 |-- movie_id: long (nullable = true)
 |-- rating: long (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- user_id: long (nullable = true)



### Carga Dataset Users.dat
Se crea el contexto SQL a traves del dataframe con los campos User_id, Movie_id, Rating, Timestamp.

In [18]:
users = sc.textFile('users.dat.txt')

El dataset esta compuesto para los siguientes campos:
UserID::Gender::Age::Occupation::Zip-code

* UserId, código del usuario
* Gender, género del usuario
* Age, rango de la edad del usuario:
    *  1:  "Menor a 18"
	* 18:  "18-24"
	* 25:  "25-34"
	* 35:  "35-44"
	* 45:  "45-49"
	* 50:  "50-55"
	* 56:  "56+"
* Occupation, profesión del usuario:
    *  0:  "other" or not specified
	*  1:  "academic/educator"
	*  2:  "artist"
	*  3:  "clerical/admin"
	*  4:  "college/grad student"
	*  5:  "customer service"
	*  6:  "doctor/health care"
	*  7:  "executive/managerial"
	*  8:  "farmer"
	*  9:  "homemaker"
	* 10:  "K-12 student"
	* 11:  "lawyer"
	* 12:  "programmer"
	* 13:  "retired"
	* 14:  "sales/marketing"
	* 15:  "scientist"
	* 16:  "self-employed"
	* 17:  "technician/engineer"
	* 18:  "tradesman/craftsman"
	* 19:  "unemployed"
	* 20:  "writer"
* zipcode, código postal donde reside el usuario.

[Fuente: http://files.grouplens.org/papers/ml-1m-README.txt]

In [19]:
users_df = users.map(lambda x: x.split("::")).map(lambda x: Row(user_id = int(x[0]),
                                     gender = str(x[1]),
                                     age_range = int(x[2]),
                                     ocupation = int(x[3]),
                                     zipcode = str(x[3]))).toDF()
users_df.show(20)

+---------+------+---------+-------+-------+
|age_range|gender|ocupation|user_id|zipcode|
+---------+------+---------+-------+-------+
|        1|     F|       10|      1|     10|
|       56|     M|       16|      2|     16|
|       25|     M|       15|      3|     15|
|       45|     M|        7|      4|      7|
|       25|     M|       20|      5|     20|
|       50|     F|        9|      6|      9|
|       35|     M|        1|      7|      1|
|       25|     M|       12|      8|     12|
|       25|     M|       17|      9|     17|
|       35|     F|        1|     10|      1|
|       25|     F|        1|     11|      1|
|       25|     M|       12|     12|     12|
|       45|     M|        1|     13|      1|
|       35|     M|        0|     14|      0|
|       25|     M|        7|     15|      7|
|       35|     F|        0|     16|      0|
|       50|     M|        1|     17|      1|
|       18|     F|        3|     18|      3|
|        1|     M|       10|     19|     10|
|       25

In [20]:
users_df.registerTempTable("Users")
users_df.printSchema()

root
 |-- age_range: long (nullable = true)
 |-- gender: string (nullable = true)
 |-- ocupation: long (nullable = true)
 |-- user_id: long (nullable = true)
 |-- zipcode: string (nullable = true)



In [21]:
users_df.select('age_range').describe().show()

+-------+------------------+
|summary|         age_range|
+-------+------------------+
|  count|              6040|
|   mean|30.639238410596025|
| stddev|12.895961726906837|
|    min|                 1|
|    max|                56|
+-------+------------------+



## Segunda tarea: Media de puntuaciones globales por año. ¿Hay algún año significativamente distinto?

Se extrae la lista de puntuaciones media globales por año, los primeros 50 registros muestran un valor medio entre 3 y 4 puntos.

In [22]:
import pyspark.sql.functions as f

In [23]:
rating_year = movies_df.join(ratings_df,on='movie_id').groupby('year').agg(f.avg('rating').alias('Puntuacion_Media')).orderBy('Puntuacion_Media', ascending=False).show(50)

+----+------------------+
|year|  Puntuacion_Media|
+----+------------------+
|1927| 4.277777777777778|
|1941| 4.179837303893086|
|1946| 4.162652980655349|
|1952| 4.126487163431434|
|1957| 4.108890214797136|
|1942| 4.103531542408693|
|1939| 4.101479915433404|
|1931| 4.097407698350353|
|1962| 4.088640973630832|
|1951|  4.08632721633497|
|1934| 4.082375478927203|
|1930|4.0811518324607325|
|1944| 4.078352180936995|
|1938| 4.072043852779953|
|1935| 4.046560846560847|
|1954| 4.045522868799656|
|1949|4.0416243654822335|
|1936|  4.04140127388535|
|1925| 4.040998217468806|
|1948| 4.029758562605278|
|1959| 4.021418174256603|
|1974|4.0082125603864736|
|1950| 4.005405405405406|
|1922| 3.991596638655462|
|1940| 3.988490182802979|
|1967|3.9859528640549398|
|1963| 3.967239185750636|
|1926|3.9639830508474576|
|1977| 3.949454673390407|
|1972| 3.946375372393247|
|1966|3.9372837370242215|
|1960|3.9265951439864484|
|1964| 3.922020484171322|
|1969| 3.921201933179239|
|1928| 3.888888888888889|
|1937|3.8858

## Tercera pregunta: ¿Cuál es la película más votada por los mayores de 60? 

Dado que el último rango de edad es de 56 a más años. Se considera la pelicula con puntuacion media mayor de este grupo de usuarios.

In [26]:
users_56= users_df.filter(users_df.age_range == 56)
movies_60_ratings_avg = ratings_df.join(users_56, on='user_id', how='inner').groupby('movie_id').agg(f.avg('rating').alias('Puntuacion_Media'),f.count('user_id').alias('Nro_Votaciones')).join(movies_df, on='movie_id')

In [27]:
movies_60_ratings_avg.select(['movie_id','movie_title','Puntuacion_media']).orderBy('Puntuacion_Media', ascending=False).show(5)

+--------+--------------------+----------------+
|movie_id|         movie_title|Puntuacion_media|
+--------+--------------------+----------------+
|    3158|Emperor and the A...|             5.0|
|     270|         Love Affair|             5.0|
|    3382|     Song of Freedom|             5.0|
|    3866|        Sunset Strip|             5.0|
|    1360|Identification of...|             5.0|
+--------+--------------------+----------------+
only showing top 5 rows



También se calcula la pelicula con mas votaciones por los usuarios.

In [28]:
movies_60_ratings_avg.select(['movie_id','movie_title','Nro_Votaciones']).orderBy('Nro_Votaciones', ascending=False).show(5)

+--------+-------------------+--------------+
|movie_id|        movie_title|Nro_Votaciones|
+--------+-------------------+--------------+
|    2858|    American Beauty|           184|
|     527|   Schindler's List|           137|
|    2396|Shakespeare in Love|           136|
|     858|     Godfather, The|           122|
|    2028|Saving Private Ryan|           121|
+--------+-------------------+--------------+
only showing top 5 rows



## Cuarta pregunta: ¿Cuál es la puntuación media de las peliculas de acción del año 2000?

Se calcula la puntuacion media por pelicula del archivo Ratings y por otro lado se obtiene las peliculas de accion del año 2000 del dataset de Movies. Ambos conjuntos de datos se une mediante un join en la columna movie_id. Se obtiene el promedio agregado de todas las puntuaciones medias.

In [29]:
rating_avg =ratings_df.groupby('movie_id').agg(f.avg('rating').alias('Puntuacion_Media')).orderBy('Puntuacion_Media', ascending=False)

In [30]:
action_movies2000_rating = movies_df.filter("category like '%Action%' AND year==2000").join(rating_avg, on='movie_id').select(['movie_title','category','year','Puntuacion_Media'])

action_movies2000_rating.agg(f.avg('Puntuacion_Media').alias('Puntuacion')).show()

+------------------+
|        Puntuacion|
+------------------+
|3.0500482715063235|
+------------------+



## Quinta pregunta: ¿ Cuál es el año en que mayor número de usuarios votaron?

Se calcula el numero de usuarios que votaron en el archivo Ratings. Este conjunto de datos se une con el dataframe de Movies. Finalmente se realiza la suma agregada por año y se muestra. El año 1999 tiene el mayor número de usuarios.

In [31]:
usr_ratings_count = ratings_df.groupby('movie_id').agg(f.count('user_id').alias('Nro_usuarios'))
movies_df.join(usr_ratings_count, on='movie_id', how='inner').groupby('year').agg(f.sum('Nro_usuarios').alias('Nro_usuarios')).orderBy('Nro_usuarios', ascending=False).show(5)

+----+------------+
|year|Nro_usuarios|
+----+------------+
|1999|       86833|
|1998|       68226|
|1997|       65402|
|1995|       60754|
|1996|       59382|
+----+------------+
only showing top 5 rows



## Sexta pregunta: ¿ Cuál es la película con mejor puntación media?

Se une el conjunto de datos de puntuaciones medias del archivo Rating con el dataframe de Movies, para obtener finalmente la lista de puntuaciones medias por pelicula.

In [32]:
movies_df.join(rating_avg, on='movie_id').select(['movie_title','Puntuacion_Media']).orderBy('Puntuacion_Media', ascending=False).show(5)

+--------------------+----------------+
|         movie_title|Puntuacion_Media|
+--------------------+----------------+
|   One Little Indian|             5.0|
|Schlafes Bruder (...|             5.0|
|       Smashing Time|             5.0|
|     Song of Freedom|             5.0|
|           Baby, The|             5.0|
+--------------------+----------------+
only showing top 5 rows

